In [ ]:
import argparse
import pathlib

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [ ]:
# set the arg parser
parser = argparse.ArgumentParser(description="PCA on a matrix")

parser.add_argument("--data_mode", type=str, default="CP", help="data mode to use")

# get the args
args = parser.parse_args()

# set data mode to either "CP" or "scDINO" or "combined" or "terminal"
data_mode = args.data_mode

In [3]:
# set the paths to the data
CP_fs_sc_profiles_path = pathlib.Path(
    "../../data/CP_feature_select/profiles/features_selected_profile.parquet"
).resolve(strict=True)
combined_profiles_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve(strict=True)
scDINO_fs_profiles_path = pathlib.Path(
    "../../data/scDINO/CLS_features_annotated_normalized_feature_selected.parquet"
).resolve(strict=True)

CP_endpoint_profiles_path = pathlib.Path(
    "../../data/CP_feature_select/endpoints/features_selected_profile.parquet"
).resolve(strict=True)

output_path = pathlib.Path("../../data/PCA/").resolve()
output_path.mkdir(parents=True, exist_ok=True)

In [4]:
if data_mode == "CP":
    # read the data
    profiles_df = pd.read_parquet(CP_fs_sc_profiles_path)
elif data_mode == "combined":
    # read the data
    profiles_df = pd.read_parquet(combined_profiles_path)
elif data_mode == "scDINO":
    # read the data
    profiles_df = pd.read_parquet(scDINO_fs_profiles_path)
elif data_mode == "terminal":
    # read the data
    profiles_df = pd.read_parquet(CP_endpoint_profiles_path)
else:
    raise ValueError("data_mode must be either 'CP' or 'scDINO' or 'combined'")
print(profiles_df.shape)
# show all columns
pd.set_option("display.max_columns", None)
profiles_df.head()

(216151, 824)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_CL_488_1,Metadata_Image_FileName_CL_488_2,Metadata_Image_FileName_CL_561,Metadata_Image_FileName_DNA,Metadata_Image_PathName_CL_488_2,Metadata_Image_PathName_CL_561,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,Cytoplasm_AreaShape_Zernike_3_3,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_4_2,Cytoplasm_AreaShape_Zernike_4_4,Cytoplasm_AreaShape_Zernike_5_1,Cytoplasm_AreaShape_Zernike_5_3,Cytoplasm_AreaShape_Zernike_5_5,Cytoplasm_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Zernike_6_2,Cytoplasm_AreaShape_Zernike_6_4,Cytoplasm_AreaShape_Zernike_6_6,Cytoplasm_AreaShape_Zernike_7_1,Cytoplasm_AreaShape_Zernike_7_3,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_7_7,Cytoplasm_AreaShape_Zernike_8_0,Cytoplasm_AreaShape_Zernike_8_2,Cytoplasm_AreaShape_Zernike_8_4,Cytoplasm_AreaShape_Zernike_8_6,Cytoplasm_AreaShape_Zernike_8_8,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_AreaShape_Zernike_9_3,Cytoplasm_AreaShape_Zernike_9_5,Cytoplasm_AreaShape_Zernike_9_7,Cytoplasm_Correlation_Correlation_CL_488_1_CL_561,Cytoplasm_Correlation_Correlation_CL_488_2_DNA,Cytoplasm_Correlation_Correlation_CL_561_DNA,Cytoplasm_Correlation_Costes_CL_488_2_CL_561,Cytoplasm_Correlation_Costes_DNA_CL_488_1,Cytoplasm_Correlation_Costes_DNA_CL_561,Cytoplasm_Correlation_K_CL_488_1_CL_488_2,Cytoplasm_Correlation_K_CL_488_2_CL_488_1,Cytoplasm_Correlation_K_CL_488_2_CL_561,Cytoplasm_Correlation_K_CL_488_2_DNA,Cytoplasm_Correlation_K_CL_561_CL_488_1,Cytoplasm_Correlation_K_CL_561_DNA,Cytoplasm_Correlation_K_DNA_CL_488_1,Cytoplasm_Correlation_K_DNA_CL_561,Cytoplasm_Correlation_Manders_CL_488_2_CL_561,Cytoplasm_Correlation_Manders_CL_488_2_DNA,Cytoplasm_Correlation_Manders_DNA_CL_488_2,Cytoplasm_Correlation_Manders_DNA_CL_561,Cytoplasm_Correlation_Overlap_CL_488_1_DNA,Cytoplasm_Correlation_Overlap_CL_488_2_DNA,Cytoplasm_Correlation_Overlap_CL_561_DNA,Cytoplasm_Correlation_RWC_CL_488_2_CL_488_1,Cytoplasm_Granularity_1_CL_488_1,Cytoplasm_Granularity_1_CL_488_2,Cytoplasm_Granularity_1_CL_561,Cytoplasm_Granularity_1_DNA,Cytoplasm_Intensity_IntegratedIntensityEdge_DNA,Cytoplasm_Intensity_IntegratedIntensity_CL_488_2,Cytoplasm_Intensity_IntegratedIntensity_CL_561,Cytoplasm_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MADIntensity_CL_488_1,Cytoplasm_Intensity_MassDisplacement_CL_488_1,Cytoplasm_Intensity_MassDisplacement_CL_561,Cytoplasm_Intensity_MassDisplacement_DNA,Cytoplasm_Intensity_MaxIntensityEdge_CL_488_2,Cytoplasm_Intensity_MaxIntensity_CL_488_2,Cytoplasm_Intensity_MeanIntensityEdge_CL_561,Cytoplasm_Intensity_MeanIntensityEdge_DNA,Cytoplasm_Intensity_MeanIntensity_DNA,Cytoplasm_Intensity_StdIntensityEdge_CL_561,Cytoplasm_Intensity_StdIntensityEdge_DNA,Cytoplasm_Location_CenterMassIntensity_X_CL_488_2,Cytoplasm_Location_CenterMassIntensity_Y_CL_488_2,Cytoplasm_RadialDistribution_FracAtD_CL_488_1_1of4,Cytoplasm_RadialDistribution_FracAtD_CL_488_1_3of4,Cytoplasm_RadialDistribution_FracAtD_CL_488_1_4of4,Cytoplasm_RadialDistribution_FracAtD_CL_561_1of4,Cytoplasm_RadialDistribution_FracAtD_CL_561_2of4,Cytoplasm_Radi

In [5]:
features_df = profiles_df.copy()
metadata_columns = [x for x in features_df.columns if "Metadata_" in x]
features_df = features_df.drop(metadata_columns, axis=1)
features_df.dropna(axis=0, inplace=True)
# pc
# separate the metadata
metadata_df = profiles_df[metadata_columns]
# fit the pca model
pca_model = PCA(n_components=2)
pca_model.fit(features_df)
# get the pca embeddings
pca_embeddings = pca_model.transform(features_df)
# create a dataframe with the pca fit and the metadata
pca_df = pd.DataFrame(pca_embeddings, columns=["PCA0", "PCA1"])
# add the metadata to the dataframe
pca_df = pd.concat([pca_df, metadata_df], axis=1)
pca_df.dropna(axis=0, inplace=True)
pca_df

,PCA0,PCA1,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_CL_488_1,Metadata_Image_FileName_CL_488_2,Metadata_Image_FileName_CL_561,Metadata_Image_FileName_DNA,Metadata_Image_PathName_CL_488_2,Metadata_Image_PathName_CL_561,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Metadata_Cells_AreaShape_BoundingBoxArea,Metadata_Nuclei_AreaShape_BoundingBoxArea
0,-7.092740,1.308090,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,2,2,2,1,1,1,2,C-09_F0002_T0001_Z0001_C02_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C03_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C04_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1166.921379,26.629744,"[1166.9213787761425, 26.629744384198297]",1,0.0,27.0,1167.0,1000002.0,-1,-1.0,"[1167.0, 27.0]",0.378511,-0.561989,0.032137
1,-6.358430,-2.461887,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,6,6,6,1,1,1,6,C-09_F0002_T0001_Z0001_C02_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C03_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C04_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1386.524390,42.020325,"[1386.5243902439024, 42.020325203252035]",2,0.0,42.0,1387.0,1000006.0,-1,-1.0,"[1387.0, 42.0]",0.476044,0.007655,0.042622
2,-8.183170,-1.604547,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,7,7,7,1,1,1,7,C-09_F0002_T0001_Z0001_C02_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C03_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C04_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,334.865122,51.944973,"[334.86512205213074, 51.94497310715764]",5,0.0,52.0,335.0,1000007.0,-1,-1.0,"[335.0, 52.0]",0.145671,-0.238557,0.475135
3,-8.028099,-1.153756,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,9,9,9,1,1,1,9,C-09_F0002_T0001_Z0001_C02_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C03_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C04_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,442.031822,105.803950,"[442.03182150694954, 105.80395025603511]",6,0.0,106.0,442.0,1000009.0,-1,-1.0,"[442.0, 106.0]",0.198615,0.154916,0.305625
4,-7.443333,0.868485,1,C-09,168,Staurosporine,39.06,positive,1,0002,0.0,10,10,10,1,1,1,10,C-09_F0002_T0001_Z0001_C02_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C03_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C04_illumcorrect.tiff,C-09_F0002_T0001_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1795.569631,101.878291,"[1795.5696313633705, 101.87829139847864]",7,0.0,102.0,1796.0,1000010.0,-1,-1.0,"[1796.0, 102.0]",0.447247,0.964563,1.202980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213272,-7.858945,0.664626,1,D-06,202,Staurosporine,4.88,test,7,0001,6.0,195,195,195,7,7,7,195,D-06_F0001_T0007_Z0001_C02_illumcorrect.tiff,D-06_F0001_T0007_Z0001_C03_illumcorrect.tiff,D-06_F0001_T0007_Z0001_C04_illumcorrect.tiff,D-06_F0001_T0007_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1601.851916,1838.7160

In [6]:
output_file_path = pathlib.Path(
    output_path, f"PCA_2D_{data_mode}_features.parquet"
).resolve()
pca_df.to_parquet(output_file_path)